<a href="https://colab.research.google.com/github/vegarab/code2seq-reproducibility-challenge/blob/feat%2Fmodel/notebooks/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code2Seq model

This notebook is used to experiment with the implementation of the code2seq model in PyTorch.

In [0]:
import torch
import torch.nn.functional as F
import torchbearer
from torchbearer import callbacks
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

This should all be moved to a global `code2seq.py` file that loads these parameters from a config-file.

In [0]:
# Grab from global config?
# based on config.py from paper implementation
DECODER_SIZE = _ 
NUM_DECODER_LAYERS = _
RNN_SIZE = _
TARGET_VOCAB_MAX_SIZE = _

This should be placed in `model.py` once final so that it can be imported. I am reluctant to move it there until it is somewhat function to make the workflow a bit easier.

In [0]:
class Encoder(nn.Module):
    def __init__(self, subtokens_input_dim, paths_input_dim):
        super(Encoder, self).__init__()

        # Pretty sure the subtokens have a single embedding even though they are shown as two in the architecture figure??
        # ... and are concated as encode_token(value(v1)) and encode_token(value(vl)) - basically encode the first and last token
        # All embeddings need to have the same dim sine they are concated before passed to the fc network?
        # Need to look at thow these are handled during the forward pass... concat?
        # z = tanh(Wh[encoder_path(v1,...,vl);encode_token(value(v1));encode_token(value(vl))])
        self.embedding_subtokens = nn.Embedding(subtokens_input_dim, EMBEDDING_DIM) 
        self.embedding_paths = nn.Embedding(paths_input_dim, EMBEDDING_DIM)

        # LSTM(input_size, hidden_size)
        # This encoder is bidirectional. Not sure what the hidden size is defined as here
        # Could be RNN_SIZE/2 from the paper implementation.. ? n in the paper?
        # Pretty sure the input_size has to follow the output of the embeddings. 
        # This is also why I think they need to be the same for the two different embeddings
        # Dropout?
        self.lstm = nn.LSTM(EMBEDDING_DIM, RNN_SIZE/2, bidirectional=True, dropout=_)

        # Linear(in_features, out_features)
        # out_features is l in the paper? or vocab size?
        # as far as I can see, they have disabled the bias in their dense layers
        self.lin = nn.Linear(RNN_SIZE/2, TARGET_VOCAB_MAX_SIZE, bias=False)

    def forward(self):
        pass

In [0]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        # DECODER_SIZE used in config. Is this k in the paper?
        # input should be the output of the FC network.. but is it still vocab size after
        # tanh activation and softmax?
        self.lstm = nn.LSTM(TARGET_VOCAB_MAX_SIZE, DECODER_SIZE, num_layers=NUM_DECODER_LAYERS)

        # as far as I can see, they have disabled the bias in their dense layers
        # out_features here is set to be the vocab size?
        self.lin = nn.Linear(DECODER_SIZE, TARGET_VOCAB_MAX_SIZE, bias=False)

    def forward(self):
        pass

This should probably be part of a `Code2Seq` class that contains an encoder and decoder?

In [0]:
def Code2Seq(nn.Module):
    def __init__(self):
        super(Code2Seq, self).__init__()
        self.encoder = Encoder(_, _)
        self.decoder = Decoder(_, _)

    def attention(self, *args, **kwargs):
        pass

    def foward(self):
        pass

The fully connected network is supposed to use a `tanh` function and `softmax` output?

$$p(y_t | y_{<t}, z_1, \dots, z_n) = \text{softmax}(W_s\tanh(W_c[c_t;h_t]))$$
$$\mathbf{\alpha^t} = \text{softmax}(h_tW_a\mathbf{z})$$